In [1]:
import imutils
import time
import dlib
import cv2
import pickle
import numpy as np
from imutils.video import VideoStream
from imutils.video import FPS

In [2]:
def convert_and_trim_bb(image, rect):
	# extract the starting and ending (x, y)-coordinates of the
	# bounding box
	startX = rect.left()
	startY = rect.top()
	endX = rect.right()
	endY = rect.bottom()
	# ensure the bounding box coordinates fall within the spatial
	# dimensions of the image
	startX = max(0, startX)
	startY = max(0, startY)
	endX = min(endX, image.shape[1])
	endY = min(endY, image.shape[0])
	# compute the width and height of the bounding box
	w = endX - startX
	h = endY - startY
	# return our bounding box coordinates
	return (startX, startY, w, h)

In [3]:
# load dlib's HOG + Linear SVM face detector
print("[INFO] loading HOG + Linear SVM face detector...")
detector = dlib.get_frontal_face_detector()

# load dlib's CNN face detector
if False:
    print("[INFO] loading CNN face detector...")
    detector = dlib.cnn_face_detection_model_v1("model/mmod_human_face_detector.dat")

# load our serialized face embedding model from disk
print("[INFO] loading face recognizer...")
embedder = cv2.dnn.readNetFromTorch("model/openface_nn4.small2.v1.t7")

# load the actual face recognition model along with the label encoder
recognizer = pickle.loads(open("output/recognizer.pk", "rb").read())
le = pickle.loads(open("output/le.pk", "rb").read())

[INFO] loading HOG + Linear SVM face detector...
[INFO] loading face recognizer...


In [4]:
# initialize the video stream, then allow the camera sensor to warm up
print("[INFO] starting video stream...")
#vs = VideoStream(src=0).start()
vs = cv2.VideoCapture(0)
time.sleep(2.0)

# vs = VideoStream(usePiCamera=True).start()

# start the FPS throughput estimator
fps = FPS().start()

# loop over frames from the video file stream
while True:
    # grab the frame from the threaded video stream
    _, frame = vs.read()
    
    # load the input image from disk, resize it, and convert it from
    # BGR to RGB channel ordering (which is what dlib expects)
    image = imutils.resize(frame, width=600)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # perform face detection using dlib's face detector
    start = time.time()
    print("[INFO[ performing face detection with dlib...")
    rects = detector(rgb, 1)
    end = time.time()
    print("[INFO] face detection took {:.4f} seconds".format(end - start))
    
    # convert the resulting dlib rectangle objects to bounding boxes,
    # then ensure the bounding boxes are all within the bounds of the
    # input image
    boxes = [convert_and_trim_bb(image, r) for r in rects]
    
    # loop over the bounding boxes
    for (x, y, w, h) in boxes:
    	# draw the bounding box on our image
    	cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    	startX, startY, endX, endY = x, y, x+w, y+h
    	face = image[startY:endY, startX:endX]
        
    	(fH, fW) = face.shape[:2]
    	# ensure the face width and height are sufficiently large
    	if fW < 20 or fH < 20:
    		continue
            
        # construct a blob for the face ROI, then pass the blob
        # through our face embedding model to obtain the 128-d
        # quantification of the face
    	faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96),
    		(0, 0, 0), swapRB=True, crop=False)
    	embedder.setInput(faceBlob)
    	vec = embedder.forward()
        
    	# perform classification to recognize the face
    	preds = recognizer.predict_proba(vec)[0]
    	j = np.argmax(preds)
    	proba = preds[j]
    	name = le.classes_[j]
        
        # draw the bounding box of the face along with the associated
        # probability
    	text = "{}: {:.2f}%".format(name, proba * 100)
    	y = startY - 10 if startY - 10 > 10 else startY + 10
    	cv2.rectangle(image, (startX, startY), (endX, endY),
    		(0, 0, 255), 2)
    	cv2.putText(image, text, (startX, y),
    		cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        
    # update the FPS counter
    fps.update()
    
    # show the output frame
    cv2.imshow("Frame", image)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
        
# stop the timer and display FPS information
fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

# do a bit of cleanup
vs.release()
cv2.destroyAllWindows()

[INFO] starting video stream...
[INFO[ performing face detection with dlib...
[INFO] face detection took 0.1629 seconds
[INFO[ performing face detection with dlib...
[INFO] face detection took 0.1720 seconds
[INFO[ performing face detection with dlib...
[INFO] face detection took 0.1476 seconds
[INFO[ performing face detection with dlib...
[INFO] face detection took 0.1424 seconds
[INFO[ performing face detection with dlib...
[INFO] face detection took 0.1438 seconds
[INFO[ performing face detection with dlib...
[INFO] face detection took 0.1402 seconds
[INFO[ performing face detection with dlib...
[INFO] face detection took 0.1414 seconds
[INFO[ performing face detection with dlib...
[INFO] face detection took 0.1400 seconds
[INFO[ performing face detection with dlib...
[INFO] face detection took 0.1432 seconds
[INFO[ performing face detection with dlib...
[INFO] face detection took 0.1404 seconds
[INFO[ performing face detection with dlib...
[INFO] face detection took 0.1409 seconds
